In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.nn.parameter import Parameter
import torchvision
from torchvision import transforms

from SpykeTorch import snn
from SpykeTorch import functional as sf
from SpykeTorch import visualization as vis
from SpykeTorch import utils
#from SpykeTorch import makeInput
#import cifar10

import numpy as np
from scipy.spatial import distance
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time

In [ ]:
rf_size = 28 # Receptive field size that will be provided as input to the column
num_neurons = rf_size*rf_size # Number of excitatory neurons in the column
startposition = 0 # Start position of the receptive field w.r.t. top left corner of image
threshold = 75 # Firing threshold for every excitatory neuron
timesteps = 8 # Resolution for timesteps and weights


#if torch.cuda.is_available():
#    torch.set_default_tensor_type('torch.cuda.FloatTensor')
#    dtype = torch.cuda.FloatTensor

In [ ]:
class PreProcTransform:
    def __init__(self, filter, timesteps = timesteps):
        self.to_tensor = transforms.ToTensor() # Convert to tensor
        self.filter = filter # Apply OnOff filtering
        self.temporal_transform = utils.Intensity2Latency(timesteps) # Convert pixel values to time
                                                    # Higher value corresponds to earlier spiketime
        self.crop = utils.Crop(startposition, rf_size) # Crop the image to form the receptive field
        
    def __call__(self, image):
        image = self.to_tensor(image) * 255
        image.unsqueeze_(0) # Adds a temporal dimension at the beginning
        image = self.filter(image)
        temporal_image = self.temporal_transform(image)
        temporal_image = temporal_image.sign() # This will create spikes
        return self.crop(temporal_image)
        
kernels = [utils.OnKernel(3), utils.OffKernel(3)]
inchannels = len(kernels)

filter = utils.Filter(kernels, padding = 2, thresholds = 50)
preproc = PreProcTransform(filter)

In [ ]:
data_root = "data"
MNIST_train = utils.CacheDataset(torchvision.datasets.MNIST(root=data_root, train=True, download=True, transform = preproc))
MNIST_test = utils.CacheDataset(torchvision.datasets.MNIST(root=data_root, train=False, download=True, transform = preproc))
MNIST_trainLoader = DataLoader(MNIST_train, batch_size=1000, shuffle=True)
MNIST_testLoader = DataLoader(MNIST_test, batch_size=1000, shuffle=True)

In [ ]:
class Column(nn.Module):
    def __init__(self):
        super(Column, self).__init__()
        # Local Convolution layer which creates columns with unique weights (NOT shared weights). The
        # number of columns is based on input_size, kernel_size and stride. Here since we are simulating
        # only one column, the input_size and kernel_size are kept same.
        self.ec = snn.HopfieldNetwork(10/128, 10/128, 24/128, 72/128, 4/128, timesteps)
        # STDP module which implements the given STDP rule for the above layer (a single column in this case)
        self.stdp = snn.ModSTDP(self.ec, 10/128, 10/128, 1/128, 96/128, 4/128, maxweight = timesteps)
        
    def forward(self, rec_field):
        ### Start of Excitatory Column ###
        out = self.ec(rec_field)
        #spike, pot = sf.fire(out, threshold, True)
        ### End of Excitatory Column ###
        ### Start of Lateral Inhibition ###
        #out = sf.pointwise_inhibition(pot).sign()
        ### End of Lateral Inhibition ###
        return out

In [ ]:
# MyColumn=Column()
# train_pattern=[torch.sum(torch.sum(MNIST_train[1][0],dim=0))]#,torch.sum(torch.sum(MNIST_train[2][0],dim=0),dim=0)]#[torch.tensor([7,0,0,0,0,7,0,0,0,0,7,0,0,0,0,7]).type(torch.FloatTensor).reshape(4,4)]
# MyColumn.ec.train_weights(train_pattern)#[torch.tensor([0,1,2,3,4,5,6,7]*2).reshape(4,4)])#[torch.sum(torch.sum(MNIST_train[0][0],dim=0),dim=0)])
# plt.imshow(train_pattern[0])#torch.tensor([0,1,2,3,4,5,6,7]*2).reshape(4,4))#torch.sum(torch.sum(MNIST_train[0][0],dim=0),dim=0))
# #MyColumn.ec.train_weights([torch.sum(torch.sum(MNIST_train[0][0],dim=0),dim=0)])
# #plt.imshow(torch.sum(torch.sum(MNIST_train[0][0],dim=0),dim=0))


MyColumn=Column()
train_pattern = []
for i in range(0,2):
    train_pattern.append(torch.sum(torch.sum(MNIST_train[i][0],dim=0),dim=0))
#train_pattern=[torch.sum(torch.sum(MNIST_train[1][0],dim=0),dim=0)]#,torch.sum(torch.sum(MNIST_train[2][0],dim=0), dim=0)]#[torch.tensor([7,0,0,0,0,7,0,0,0,0,7,0,0,0,0,7]).type(torch.FloatTensor).reshape(4,4)]
#train_pattern is 1x28x28
MyColumn.ec.train_weights(train_pattern)

noSTDPcolumn = Column()
noSTDPcolumn.ec.train_weights(train_pattern)
#plt.imshow(train_pattern[4])

In [ ]:
plt.figure(figsize=(6, 5))
w_mat = plt.imshow(MyColumn.ec.W, cmap=cm.coolwarm)
plt.colorbar(w_mat)
plt.title("Network Weights")
plt.tight_layout()
plt.savefig("weights.png")
plt.show()

init_weights = MyColumn.ec.W.clone()

out=[]
weights=[]
for i in range(100):
    pattern=[torch.sum(MNIST_train[26][0],dim=1).reshape(8,784).clone(),torch.sum(MNIST_train[31][0],dim=1).reshape(8,784).clone()]
    #torch.tensor([0]*784).type(torch.FloatTensor)#torch.tensor([2,6,3,1,7,4,5,2,7,7,7,7,3,7,5,5]).type(torch.FloatTensor)#(torch.tensor([0,1,2,3,4,5,6,7]*2).type(torch.FloatTensor))#torch.sum(torch.sum(MNIST_train[1][0],dim=0),dim=0).reshape(784)-4)/4
    out.append(MyColumn.ec.forward(pattern,threshold=100,num_iter=20).clone())
    weights.append(MyColumn.ec.W.clone())

In [ ]:
outSTDP=[]
out = []
for i in range(0,2):
    testPattern = []
    testPattern.append(torch.sum(MNIST_train[i][0],dim=1).reshape(8,784).clone())
    out.append(noSTDPcolumn.ec.forward(testPattern,threshold=100,num_iter=20, noSTDP=True).clone())

for j in range(0,40):
    array = []
    for i in range(0,2):
        testPattern = []
        testPattern.append(torch.sum(MNIST_train[i][0],dim=1).reshape(8,784).clone())
        array.append(MyColumn.ec.forward(testPattern,threshold=100,num_iter=20).clone())
    outSTDP.append(array)

In [ ]:
it = 40

for i in range(0,2):
    plt.subplot(2,2,i+1), plt.imshow(train_pattern[i])
    plt.subplot(2,2,2+1+i), plt.imshow(torch.sum(outSTDP[it-1][i].squeeze(0),dim=0))
plt.savefig("two_zeroWeightInit_"+ str(it)+ ".png")

test = [torch.sum(MNIST_train[31][0],dim=1).reshape(8,784)]
plt.imshow(torch.sum(MyColumn.ec.forward(test,threshold=100,num_iter=20).squeeze(0),dim=0))

plt.imshow(torch.sum(torch.sum(MNIST_train[31][0],dim=1),dim=0))

In [ ]:
it = 16
for i in range(0,2):
    plt.subplot(3,2,i+1), plt.imshow(train_pattern[i])
    plt.subplot(3,2,2+1+i), plt.imshow(torch.sum(out[i].squeeze(0),dim=0))
    plt.subplot(3,2,4+1+i), plt.imshow(torch.sum(outSTDP[it-1][i].squeeze(0),dim=0))
plt.savefig("two_heb_"+ str(it)+".png")

In [ ]:
plt.imshow(torch.sum(outSTDP[69][2].squeeze(0),dim=0))

In [ ]:
plt.imshow(torch.sum(outSTDP[19][1].squeeze(0),dim=0))


In [ ]:
plt.imshow(train_pattern[1])

In [ ]:
np.unique(torch.sum(out.squeeze(0),dim=0))

In [ ]:
np.unique(torch.sum(pattern,dim=0).reshape(28,28))

In [ ]:
pattern

In [ ]:
distance.hamming(out.reshape(784),(train_pattern[0]-4).reshape(784))

In [ ]:
distance.hamming(out.reshape(784),(pattern-4))

In [ ]:
out.squeeze(0).shape

In [ ]:
np.unique(MyColumn.ec.W)

In [ ]:
out.shape

In [ ]:
plt.imshow(init_weights-weights[95],cmap=cm.coolwarm)

In [ ]:
plt.imshow(MyColumn.ec.W,cmap=cm.coolwarm)

In [ ]:
torch.max(MyColumn.ec.W)

In [ ]:
torch.min(MyColumn.ec.W)

In [ ]:
train_pattern

In [ ]:
plt.imshow(torch.sum(out[0].squeeze(0),dim=0)-torch.sum(out[99].squeeze(0),dim=0))

In [ ]:
plt.imshow(torch.sum(torch.sum(MNIST_train[19][0],dim=1),dim=0).reshape(28,28))